In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
#import datetime 

from matplotlib import pyplot
from math import sqrt
from numpy import median, std, mean, array, concatenate
from pandas import DataFrame, concat, read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Flatten, Masking, Dropout, ConvLSTM2D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
#load data and arrange to correct format
df = pd.read_csv("return_matrix.csv")
df = df.transpose() 
df.to_csv('workablefile.csv')
df = pd.read_csv('workablefile.csv', index_col=0)
data = df.values
#normalize all non-nan data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

#change threshhold for identifying nan's, higher (closer to zero) value decreases nan identification accuracy, 
#but incentives the model to focus on the other data
data = np.nan_to_num(data, nan=-0.01)

C:\Python38\lib\site-packages\sklearn\preprocessing\_data.py:373: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Python38\lib\site-packages\sklearn\preprocessing\_data.py:374: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [3]:
#some configs
n_steps = 12 #using last 12 months data(t-12) to predict current month (t)
n_stocks = len(df.columns)
n_variables = n_steps*n_stocks
n_test = len(df.values)-132 # this gives a 10y burn in period? #n_train = len(df.values)-n_test
n_records = len(data)

In [4]:
#change the data to a time-series format
def series_to_supervised(data, n_in=n_steps, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
#calling the above function
data = series_to_supervised(data)

In [5]:
#split data into training/testin and inputs/outputs
def train_test_split(dataset, n_test):
    return data.values[:n_test, :], data.values[n_test:, :]
def input_output(train, test):
    return train[:, :n_steps*n_stocks], train[:,-n_stocks:], test[:, :n_steps*n_stocks], test[:,-n_stocks:]

##Calling the above functions
test, train = train_test_split(data.values, n_test)
train_x, train_y, test_x, test_y = input_output(train,test)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60, 15516) (60, 1293) (78, 15516) (78, 1293)


In [6]:
#fit the model
def model_fit(train_x, train_y):
    #define the model
    model = Sequential()
    model.add(LSTM(1250, activation='relu', return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(1250, activation='relu', return_sequences=True))
    model.add(LSTM(1250, activation='relu', return_sequences=True))
    model.add(LSTM(1250, activation='relu'))
    model.add(Dense(n_stocks))
    model.compile(loss='mse', optimizer='Adam')
    # fit network
    model.fit(train_x, train_y, epochs=100, batch_size=3, verbose=1, shuffle=False)
    return model

In [7]:
#forecast with the fit model
def model_predict(model, history):
    #fix shape of input then predict y
    x_input = history.reshape((history.shape[0], 1, history.shape[-1]))
    yhat = model.predict(x_input, verbose=2)
    return yhat[0]

In [8]:
# root mean squared error or rmse
def measure_rmse(test_y, predictions):
    return mean_squared_error(test_y, predictions)

In [9]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    test, train = train_test_split(data, n_test)
    train_x, train_y, test_x, test_y = input_output(train,test)
    train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[-1]))
    test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[-1]))
    # fit model
    #model = model_fit(train_x, train_y)
    # seed history with training dataset and step over each time-step in the test set and append
    history = train_x
    for i in range(len(test_x)):
        # fit model using the training data
        model = model_fit(history, train_y)
        yhat = model_predict(model, history)
        predictions.append(yhat)
        # add the observed valus to the data
        history = np.append([history],[test_x[i]])
        history = history.reshape((train_x.shape[0]+1, 1, train_x.shape[-1]))
        history = np.delete(history, 0, axis=0)
        print(i)
    # estimate prediction error
    yhat = predictions
    error = measure_rmse(test_y, predictions)
    print(' > %.3f' % error)
    return error, yhat

In [10]:
#zztest = walk_forward_validation(data, n_test)

error, yhat_test = walk_forward_validation(data, n_test)

Epoch 1/3
60/60 [==============================] - 2s 42ms/step - loss: 0.0442
Epoch 2/3
60/60 [==============================] - 1s 19ms/step - loss: 0.0193
Epoch 3/3
60/60 [==============================] - 1s 20ms/step - loss: 0.0178


KeyboardInterrupt: 

In [ ]:
print(error)
#print(yhat_test)
yhat_test = DataFrame(yhat_test)
print(yhat_test.shape)

In [ ]:
#print(mean(scores), std(scores))

In [ ]:
print(yhat_test)


In [ ]:
#yhat_test = model.predict(test_x)
#yhat_train = model.predict(train_x)
#yhat = yhat.transpose()
#test_X = test_x.reshape((test_x.shape[0], test_x.shape[2]))

#change all negative values to NaN (all features were between 0-1 while NaNs were -1)
yhat_test = DataFrame(yhat_test)
#yhat_test[yhat_test < 0] = np.NaN

test_y = DataFrame(test_y)
test_y[test_y < 0] = np.NaN

#yhat_train = DataFrame(yhat_train)
#yhat_train[yhat_train < 0] = np.NaN

#train_y = DataFrame(train_y)
#train_y[train_y < 0] = np.NaN

#invert normalization (for the predicted values)
#yhat_train = scaler.inverse_transform(yhat_train)
#train_y = scaler.inverse_transform(train_y)

yhat_test = scaler.inverse_transform(yhat_test)
test_y = scaler.inverse_transform(test_y)
#print(yhattpd)

## save predicted and actual values to csv
yhat_test = DataFrame(yhat_test)
yhat_test.to_csv('yhat_test.csv')
test_y = DataFrame(test_y)
test_y.to_csv('y_test.csv')